In [3]:
import psycopg2
import pandas as pd

In [1]:
def create_database():
    # connect to default database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=postgreSQL")
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    #Create Sparkify Database with UTF-8 encoding
    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")

    #Close connection to the default database
    conn.close()

    #Connect to the accounts database
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=postgreSQL")
    cur = conn.cursor()
    return cur, conn

In [5]:
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [6]:
def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

### Accounts Country DataFrame

In [14]:
AccountsCountry = pd.read_csv("AccountData/Wealth-AccountsCountry.csv")

#### Verifying the data was recieved

In [15]:
AccountsCountry.head()

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


#### Extracting the necessary data from the DataFrame and verifying it was done properly

In [51]:
AccountsCountry_Clean = AccountsCountry[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']].copy()

In [52]:
AccountsCountry_Clean.rename(columns = {'Code':'Country_Code'}, inplace=True)

In [53]:
AccountsCountry_Clean = AccountsCountry_Clean.drop_duplicates(subset=['Country_Code'], keep='first')

In [54]:
AccountsCountry_Clean = AccountsCountry_Clean.dropna(how='all')

In [59]:
AccountsCountry_Clean.head()

,Country_Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


### Accounts Data DataFrame

In [83]:
AccountsData = pd.read_csv("AccountData/Wealth-AccountData.csv")

In [84]:
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6353.737,5466.692,5901.072,6492.73,6872.019,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.1,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4737.857,4059.321,4191.445,4536.972,4695.071,...,3.529144e+03,3.613606e+03,3.603694e+03,3.683810e+03,3.696744e+03,3.719608e+03,3.767628e+03,3872.142,3992.188,4161.137


In [85]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995 [YR1995]', '1996 [YR1996]', '1997 [YR1997]', '1998 [YR1998]',
       '1999 [YR1999]', '2000 [YR2000]', '2001 [YR2001]', '2002 [YR2002]',
       '2003 [YR2003]', '2004 [YR2004]', '2005 [YR2005]', '2006 [YR2006]',
       '2007 [YR2007]', '2008 [YR2008]', '2009 [YR2009]', '2010 [YR2010]',
       '2011 [YR2011]', '2012 [YR2012]', '2013 [YR2013]', '2014 [YR2014]',
       '2015 [YR2015]', '2016 [YR2016]', '2017 [YR2017]', '2018 [YR2018]'],
      dtype='object')

In [86]:
AccountsData_Clean = AccountsData[['Country Name', 'Country Code', 'Series Name', 'Series Code', '1995 [YR1995]', \
                                   '2000 [YR2000]', '2005 [YR2005]', '2010 [YR2010]', '2014 [YR2014]']]

In [87]:
AccountsData_Clean.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],2000 [YR2000],2005 [YR2005],2010 [YR2010],2014 [YR2014]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.36E+10,5.440000e+10,6.810000e+10,7.250000e+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,14118.13,1.807230e+04,2.336286e+04,2.509002e+04
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3424.007,2.222115e+03,2.889150e+03,2.957420e+03
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6872.019,1.168935e+04,1.578992e+04,1.699138e+04
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4695.071,2.886736e+03,3.613606e+03,3.719608e+03


### Creation of Account Series DataFrame

In [94]:
AccountSeries = pd.read_csv("AccountData/Wealth-AccountSeries.csv")

In [95]:
AccountSeries.head()

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [96]:
AccountSeries_Clean = AccountSeries[['Code', 'Topic', 'Indicator Name']]

In [97]:
AccountSeries_Clean.head()

,Code,Topic,Indicator Name
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$)
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$)
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con..."
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const..."
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201..."


### Connecting to accounts database and creating tables

In [71]:
cur, conn = create_database()

In [72]:
accounts_country_table = ("""CREATE TABLE IF NOT EXISTS accountsCountry(
Country_Code VARCHAR PRIMARY KEY,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
)""")

In [73]:
cur.execute(accounts_country_table)
conn.commit()

In [74]:
accounts_data_table = ("""CREATE TABLE IF NOT EXISTS accountsData(
Country_Name VARCHAR,
Country_Code VARCHAR,
indicator_name VARCHAR,
indicator_code VARCHAR,
year_1995 numeric,
year_2000 numeric,
year_2005 numeric,
year_2010 numeric,
year_2014 numeric
)""")
cur.execute(accounts_data_table)
conn.commit()

In [75]:
account_series_table = ("""CREATE TABLE IF NOT EXISTS accountSeries(
series_code VARCHAR,
topic VARCHAR,
indicator_code VARCHAR
)""")
cur.execute(account_series_table)
conn.commit()

### Inserting Data into Tables

In [76]:
accounts_country_insert = ("""INSERT INTO accountsCountry(
Country_Code,
short_name,
table_name,
long_name,
currency_unit)
VALUES (%s, %s, %s, %s, %s);
""")

In [77]:
for i, row in AccountsCountry_Clean.iterrows():
    print(list(row))
    cur.execute(accounts_country_insert, list(row))

['ALB', 'Albania', 'Albania', 'Republic of Albania', 'Albanian lek']
['ARG', 'Argentina', 'Argentina', 'Argentine Republic', 'Argentine peso']
['ARM', 'Armenia', 'Armenia', 'Republic of Armenia', 'Armenian dram']
['AUS', 'Australia', 'Australia', 'Commonwealth of Australia', 'Australian dollar']
['AUT', 'Austria', 'Austria', 'Republic of Austria', 'Euro']
['AZE', 'Azerbaijan', 'Azerbaijan', 'Republic of Azerbaijan', 'New Azeri manat']
['BHR', 'Bahrain', 'Bahrain', 'Kingdom of Bahrain', 'Bahraini dinar']
['BGD', 'Bangladesh', 'Bangladesh', "People's Republic of Bangladesh", 'Bangladeshi taka']
['BLR', 'Belarus', 'Belarus', 'Republic of Belarus', 'Belarusian rubel']
['BEL', 'Belgium', 'Belgium', 'Kingdom of Belgium', 'Euro']
['BLZ', 'Belize', 'Belize', 'Belize', 'Belize dollar']
['BEN', 'Benin', 'Benin', 'Republic of Benin', 'West African CFA franc']
['BOL', 'Bolivia', 'Bolivia', 'Plurinational State of Bolivia', 'Bolivian Boliviano']
['BIH', 'Bosnia and Herzegovina', 'Bosnia and Herzego

In [81]:
accounts_data_insert = ("""INSERT INTO accountsData(
Country_Name,
Country_Code,
indicator_name,
indicator_code,
year_1995,
year_2000,
year_2005,
year_2010,
year_2014)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
""")

In [89]:
for i, row in AccountsData_Clean.iterrows():
    print(list(row))
    cur.execute(accounts_data_insert, list(row))

['Albania', 'ALB', 'Human capital (constant 2018 US$)', 'NW.HCA.TO', '4.49E+10', '4.36E+10', 54400000000.0, 68100000000.0, 72500000000.0]
['Albania', 'ALB', 'Human capital per capita (constant 2018 US$)', 'NW.HCA.PC', '14072.79', '14118.13', 18072.3, 23362.86, 25090.02]
['Albania', 'ALB', 'Human capital per capita, employed female (constant 2018 US$)', 'NW.HCA.FEMP.PC', '3468.858', '3424.007', 2222.115, 2889.15, 2957.42]
['Albania', 'ALB', 'Human capital per capita, employed male (constant 2018 US$)', 'NW.HCA.MEMP.PC', '6507.931', '6872.019', 11689.35, 15789.92, 16991.38]
['Albania', 'ALB', 'Human capital per capita, female (constant 2018 US$)', 'NW.HCA.FEMA.PC', '4893.006', '4695.071', 2886.736, 3613.606, 3719.608]
['Albania', 'ALB', 'Human capital per capita, male (constant 2018 US$)', 'NW.HCA.MALE.PC', '9179.78', '9423.057', 15185.56, 19749.26, 21370.41]
['Albania', 'ALB', 'Human capital per capita, self-employed female (constant 2018 US$)', 'NW.HCA.FSEM.PC', '1424.149', '1271.063',

InvalidTextRepresentation: invalid input syntax for type numeric: ".."
LINE 11: ...sets: oil (constant 2018 US$)', 'NW.NCA.SAOI.PC', '..', '..'...
                                                              ^


In [105]:
account_series_insert= ("""INSERT INTO accountSeries(
series_code,
topic,
indicator_code)
VALUES (%s, %s, %s)
""")

In [106]:
for i, row in AccountSeries_Clean.iterrows():
    print(list(row))
    cur.execute(account_series_insert, list(row))

['NW.HCA.TO', 'Human capital', 'Human capital (constant 2018 US$)']
['NW.HCA.PC', 'Human capital', 'Human capital per capita (constant 2018 US$)']
['NW.HCA.FEMP.PC', 'Human capital', 'Human capital per capita, employed female (constant 2018 US$)']
['NW.HCA.MEMP.PC', 'Human capital', 'Human capital per capita, employed male (constant 2018 US$)']
['NW.HCA.FEMA.PC', 'Human capital', 'Human capital per capita, female (constant 2018 US$)']
['NW.HCA.MALE.PC', 'Human capital', 'Human capital per capita, male (constant 2018 US$)']
['NW.HCA.FSEM.PC', 'Human capital', 'Human capital per capita, self-employed female (constant 2018 US$)']
['NW.HCA.MSEM.PC', 'Human capital', 'Human capital per capita, self-employed male (constant 2018 US$)']
['NW.HCA.FEMP.TO', 'Human capital', 'Human capital, employed female (constant 2018 US$)']
['NW.HCA.MEMP.TO', 'Human capital', 'Human capital, employed male (constant 2018 US$)']
['NW.HCA.FEMA.TO', 'Human capital', 'Human capital, female (constant 2018 US$)']
['

In [107]:
conn.commit()

In [108]:
conn.close()
cur.close()